In [1]:
import pandas as pd
from src.constantes import columnas_relevantes_eph

In [2]:
pd.set_option('display.max_colwidth', 3000)

In [3]:
df = pd.read_csv("datos-eph/usu_individual_T323.txt", sep=";")[columnas_relevantes_eph]

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_4484\2674065315.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datos-eph/usu_individual_T323.txt", sep=";")[columnas_relevantes_eph]


In [13]:
import pandas as pd
import plotly.express as px
P21_column = df['P21']

# Create a histogram to visualize the distribution
fig = px.histogram(df, x='P21', nbins=100, title='Distribution of Column P21')
fig.update_layout(xaxis_title='P21 Values', yaxis_title='Frequency', template = None)
fig.show()